In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# Import libraries
import pandas as pd
import os
import yaml
from pathlib import Path
import numpy as np

from FindConfThr import (
    load_and_prepare_model,
    predict_images,
    load_yolo_labels,
    yolo_to_bbox,
    convert_df,
    get_preds_data,
    evaluate_detection,
    compute_map,
)

In [ ]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

with open("data.yaml", "r") as f:
	data = yaml.safe_load(f)
class_map = {
    name: i for i, name in enumerate(data['names'])
}

In [ ]:
from glob import glob

latest_run_dir = sorted(glob("zindi_challenge_cacao_stage2/train*"), key=lambda x: int(x.split('train')[-1]))[-1]

# Validate the model on the validation set
BEST_PATH = f"{latest_run_dir}/weights/best.pt"
# BEST_PATH = 'zindi_challenge_cacao_stage2/train10/weights/best.pt'
BEST_PATH

In [ ]:
# Validate the model on the validation set
BEST_CFG = f"{latest_run_dir}/args.yaml"
# BEST_CFG = 'zindi_challenge_cacao_stage2/train8/args.yaml'
BEST_CFG

In [ ]:
# Load the trained YOLO model
batch_size=16
model, cfg = load_and_prepare_model(
    BEST_PATH, BEST_CFG, batch_size
)

In [ ]:
# Convert the list to a DataFrame for all images
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR
sub = predict_images(model, test_dir_path, cfg, batch_size)

In [ ]:
sub.head()

In [ ]:
sub.describe()

In [ ]:
sub['class'].value_counts()

In [ ]:
# Example usage
if os.path.exists('dataset/labels.csv'):
	labels = pd.read_csv('dataset/labels.csv')
else:
	label_folder = VAL_LABELS_DIR
	labels = load_yolo_labels(label_folder)
	labels.to_csv('dataset/labels.csv', index=False)
labels.sample(5)


# Example usage
if os.path.exists('dataset/converted_labels.csv'):
	converted_labels = pd.read_csv('dataset/converted_labels.csv')
else:
	converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
	converted_labels.to_csv('dataset/converted_labels.csv', index=False)
converted_labels.sample(5)

In [ ]:
converted_labels.describe()

In [ ]:
converted_labels["Image_ID"].nunique()

In [ ]:
converted_labels['class_id'].value_counts()

In [ ]:
class_map

In [ ]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

In [ ]:
converted_labels.sample(5)

In [ ]:
sub.sample(5)

In [ ]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

In [ ]:
converted_labels.isna().sum()

In [ ]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

In [ ]:
sub.sample(3)

In [ ]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

In [ ]:
import FindConfThr

FindConfThr.ground_truth_df = converted_labels

In [ ]:
predictions = get_preds_data(sub, None)

len(predictions)

In [ ]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

In [ ]:
converted_labels.sample(5)

In [ ]:
preds = list(get_preds_data(sub, 0).values())
targets = list(ground_truth.values())

results = compute_map(preds, targets, cfg)
print("mAP Results:", 0, " - ", results)

In [ ]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [ ]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

In [ ]:
model.overrides